In [3]:
class Connector :
    def __init__(self, owner, name, activates=0, monitor=0):
        self.value = None
        self.owner = owner
        self.name  = name
        self.monitor  = monitor
        self.connects = []
        self.activates= activates
    def connect(self, inputs):
        if not isinstance(inputs, list):
            inputs = [inputs]
        for input in inputs:
            self.connects.append(input)
    def set(self, value):
        if self.value == value:
            return      # Ignore if no change
        self.value = value
        if self.activates:
            self.owner.evaluate()
        for con in self.connects:
            con.set(value)
    

In [4]:
class LC:
    # Logic Circuits have names and an evaluation function defined in child
    # classes. They will also contain a set of inputs and outputs.
    def __init__(self, name):
        self.name = name

    def evaluate(self):
        return

In [5]:
class Gate2(LC):         # two input gates. Inputs A and B. Output C.
    def __init__(self, name):
        LC.__init__(self, name)
        self.A = Connector(self, 'A', activates=1)
        self.B = Connector(self, 'B', activates=1)
        self.C = Connector(self, 'C')

In [6]:
class Xor(Gate2):
    def __init__(self, name):
        Gate2.__init__(self, name)
        self.A1 = And("A1")  # See circuit drawing to follow connections
        self.A2 = And("A2")
        self.I1 = Not("I1")
        self.I2 = Not("I2")
        self.O1 = Or("O1")
        self.A.connect([self.A1.A, self.I2.A])
        self.B.connect([self.I1.A, self.A2.A])
        self.I1.B.connect([self.A1.B])
        self.I2.B.connect([self.A2.B])
        self.A1.C.connect([self.O1.A])
        self.A2.C.connect([self.O1.B])
        self.O1.C.connect([self.C])

In [7]:
class And(Gate2):       # two input AND Gate
    def __init__(self, name):
        Gate2.__init__(self, name)

    def evaluate(self):
        self.C.set(self.A.value and self.B.value)


class Or(Gate2):         # two input OR gate.
    def __init__(self, name):
        Gate2.__init__(self, name)

    def evaluate(self):
        self.C.set(self.A.value or self.B.value)

In [8]:
class Not(LC):         # Inverter. Input A. Output B.
    def __init__(self, name):
        LC.__init__(self, name)
        self.A = Connector(self, 'A', activates=1)
        self.B = Connector(self, 'B')

    def evaluate(self):
        self.B.set(not self.A.value)

In [9]:
class HalfAdder(LC):         # One bit adder, A,B in. Sum and Carry out
    def __init__(self, name):
        LC.__init__(self, name)
        self.A = Connector(self, 'A', 1)
        self.B = Connector(self, 'B', 1)
        self.S = Connector(self, 'S')
        self.C = Connector(self, 'C')
        self.X1 = Xor("X1")
        self.A1 = And("A1")
        self.A.connect([self.X1.A, self.A1.A])
        self.B.connect([self.X1.B, self.A1.B])
        self.X1.C.connect([self.S])
        self.A1.C.connect([self.C])

In [10]:
class FullAdder(LC):         # One bit adder, A,B,Cin in. Sum and Cout out
    def __init__(self, name):
        LC.__init__(self, name)
        self.A = Connector(self, 'A', 1, monitor=1)
        self.B = Connector(self, 'B', 1, monitor=1)
        self.Cin = Connector(self, 'Cin', 1, monitor=1)
        self.S = Connector(self, 'S', monitor=1)
        self.Cout = Connector(self, 'Cout', monitor=1)
        self.H1 = HalfAdder("H1")
        self.H2 = HalfAdder("H2")
        self.O1 = Or("O1")
        self.A.connect([self.H1.A])
        self.B.connect([self.H1.B])
        self.Cin.connect([self.H2.A])
        self.H1.S.connect([self.H2.B])
        self.H1.C.connect([self.O1.B])
        self.H2.C.connect([self.O1.A])
        self.H2.S.connect([self.S])
        self.O1.C.connect([self.Cout])

In [19]:
def test8Bit(a, b):  
    F0 = FullAdder("F0")
    F1 = FullAdder("F1")
    F0.Cout.connect(F1.Cin)
    F2 = FullAdder("F2")
    F1.Cout.connect(F2.Cin)
    F3 = FullAdder("F3")
    F2.Cout.connect(F3.Cin)
    F4 = FullAdder("F4")
    F3.Cout.connect(F4.Cin)
    F5 = FullAdder("F5")
    F4.Cout.connect(F5.Cin)
    F6 = FullAdder("F6")
    F5.Cout.connect(F6.Cin)
    F7 = FullAdder("F7")
    F6.Cout.connect(F7.Cin)
        
    F0.Cin.set(0)
    F0.A.set(bit(a, 7))
    F0.B.set(bit(b, 7)) 
    F1.A.set(bit(a, 6))
    F1.B.set(bit(b, 6))
    F2.A.set(bit(a, 5))
    F2.B.set(bit(b, 5))
    F3.A.set(bit(a, 4))
    F3.B.set(bit(b, 4))
    F4.A.set(bit(a, 3))
    F4.B.set(bit(b, 3))
    F5.A.set(bit(a, 2))
    F5.B.set(bit(b, 2))
    F6.A.set(bit(a, 1))
    F6.B.set(bit(b, 1))
    F7.A.set(bit(a, 0))
    F7.B.set(bit(b, 0))
    
    byte = [F7.Cout.value, F7.S.value,F6.S.value,F5.S.value,
                                   F4.S.value,F3.S.value,F2.S.value,F1.S.value,F0.S.value]
    return byte

In [12]:
def bit(x, bit):
    return x[bit] == '1'

In [13]:
def isZero(a):
    Out = Not("Out")
    def isSimpleZero(a):
        if len(a) == 2:
            F0 = Or("F0")
            F0.A.set(bit(a, 0))
            F0.B.set(bit(a, 1))
            return F0.C.value
        else:
            F1 = Or("F1")
            left = isSimpleZero(a[0:int(len(a)/2)])
            right = isSimpleZero(a[int(len(a)/2):int(len(a))])
            F1.A.set(left)
            F1.B.set(right)
            # print(left, right)
            return (F1.C.value)
            
    Out.A.set(isSimpleZero(a))
    Out.evaluate()
    return str(int(Out.B.value))


In [14]:
def isBasicZero(a):
    F0 = Or("F0")
    F1 = Or("F1")
    F2 = Or("F2")
    F3 = Or("F3")

    G0 = Or("G6")
    G1 = Or("G7")

    H0 = Or("G7")

    Cout = Not("Salida")

    F0.A.set(bit(a, 7))
    F0.B.set(bit(a, 6))

    F1.A.set(bit(a, 5))
    F1.B.set(bit(a, 4))
    
    F2.A.set(bit(a, 3))
    F2.B.set(bit(a, 2))

    F3.A.set(bit(a, 1))
    F3.B.set(bit(a, 0))
#------------------------------
    G0.A.set(F0.C.value)
    G0.B.set(F1.C.value)

    G1.A.set(F2.C.value)
    G1.B.set(F3.C.value)

#---------------------------
    H0.A.set(G0.C.value)
    H0.B.set(G1.C.value)

    Cout.A.set(H0.C.value)
    Cout.evaluate()

    return(Cout.B.value)


In [15]:
isBasicZero('00000000')

True

In [16]:
def isNegative(a):
    F0 = And("F0")
    F0.A.set(bit(a, 0))
    F0.B.set(True)
    return(str(int(F0.C.value)))


In [17]:
isNegative("00001000")

'0'

In [20]:
test8Bit('11100010','10101010')

[True, True, False, False, False, True, True, False, False]

In [ ]:
import unittest
class Test8Bits(unittest.TestCase):

    def test1(self):
        self.assertEqual(test8Bit('11000000','10101000'),[True, False, True, True, False, True, False, False, False]), "Should be True False True True False True False False False"
    def test2(self):
        self.assertEqual(test8Bit('11000010','00101010'),[False, True, True, True, False, True, True, False, False]), "Should be True False True True False True False False False"
    def test3(self):
        self.assertEqual(test8Bit('11000010','00101010'),[True, True, False, False, False, True, True, False, False]), "Should be True False True True False True False False False"
    def test4(self):
        self.assertEqual(test8Bit('11000010','00101010'),[True, False, False, False, False, True, True, False, False]), "Should be True False True True False True False False False"
    def test5(self):
        self.assertEqual(test8Bit('01100010','10101011'),[True, False, False, False, False, True, True, False, True]), "Should be True False True True False True False False False"
        

unittest.main(argv=[''], verbosity=2, exit=False)

In [ ]:
import unittest
class TestIsNegative(unittest.TestCase):

    def test1(self):
        self.assertEqual(isNegative('11000000'),'1'), "Should be True"
    def test2(self):
        self.assertEqual(isNegative('01000010'),'0'), "Should be False"
    def test3(self):
        self.assertEqual(isNegative('11000010'), '1'), "Should be True"
    def test4(self):
        self.assertEqual(isNegative('11111111'), '1'), "Should be True"
    def test5(self):
        self.assertEqual(isNegative('00000000'),'0'), "Should be False"

unittest.main(argv=[''], verbosity=2, exit=False)